In [2]:
#  !pip install openai requests python-dotenv

from dotenv import load_dotenv

# At first, add your api key to .env file as DEV_GENAI_API_KEY='Insert_your_Dev_GenAI_Text_to_Text_API_key_here' and load env file
load_dotenv('.env', override=True)

# alternatively you can export an enviroment variable using following command: export DEV_GENAI_API_KEY="Insert_your_Dev_GenAI_Text_to_Text_API_key_here" 

from openai import OpenAI
import httpx
import os

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
http_client=httpx.Client(verify=False)
client = OpenAI(
    base_url='https://genai-api-dev.dell.com/v1',
    http_client=http_client,
    api_key=os.environ["DEV_GENAI_API_KEY"]
)


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json
import requests
import os

streaming = True
max_output_tokens = 200

# Available Models list
available_models = ["mixtral-8x7b-instruct-v01", "llamaguard-7b", "mistral-7b-instruct-v03", "phi-3-mini-128k-instruct", "llama-3-8b-instruct", "llama-3-1-8b-instruct"]

# Let's select the model from available list
model_selected = available_models[0]

def stream_and_yield_response(response):
    for chunk in response.iter_lines():
        decoded_chunk = chunk.decode("utf-8")
        if decoded_chunk == "data: [DONE]":
            pass
        elif decoded_chunk.startswith("data: {"):
            payload = decoded_chunk.lstrip("data:")
            json_payload = json.loads(payload)
            yield json_payload['choices'][0]['text']


# function from confluence
def llm_api(data):
    """
    Create a request to Dev GenAI Text to Text model with API key in header.
    """
    
    url = f"https://genai-api-dev.dell.com/v1/completions"
      
    headers = {
        'accept': 'application/json',
        'api-key': os.environ["DEV_GENAI_API_KEY"],
        'Content-Type': 'application/json'
    }
      
    try:
        response = requests.post(url, headers=headers, json=data, stream=data['stream'], verify=False)
        response.raise_for_status()
        
        if data['stream']:
            for result in stream_and_yield_response(response):
                print(result, end='')
        else:
            response_dict = response.json()
            result = response_dict['choices'][0]['text']
            print(result)
           
    except requests.exceptions.HTTPError as err:
        print('Error code:', err.response.status_code)
        print('Error message:', err.response.text)
    except Exception as err:
        print('Error:', err)

# Model instruction and Parameters
instruction_text = f'Can you explain who are the Los Angeles Dodgers and what are they known for is in less than {max_output_tokens} tokens?'
  
data = {
    'prompt': instruction_text,
    'temperature': 0.5,
    'top_p': 0.95,
    'max_tokens': max_output_tokens,
    'stream': streaming,
    'model': model_selected
    }

# API Call
llm_api(data)

c:\Users\Ashish_Sharma30\OneDrive - Dell Technologies\Desktop\Self Learning project\my_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-api-dev.dell.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(




The Los Angeles Dodgers are a Major League Baseball team based in Los Angeles, California. Established in 1883, they were originally called the Brooklyn Atlantics and later became the Brooklyn Dodgers before moving to Los Angeles in 1958. The Dodgers are known for their rich history, numerous Hall of Fame players, and winning tradition. They have won six World Series titles, 23 National League pennants, and 12 division titles. The team is also known for its iconic Dodger Stadium, which opened in 1962 and is the third-oldest ballpark in Major League Baseball. The Dodgers have a large and passionate fanbase, and their rivalry with the San Francisco Giants is one of the most storied in sports.